In [1]:
import sys
sys.path.insert(0, 'F:/Workspace/idx2/build/Source/Python/Release')
sys.path.insert(0, 'D:/pascucci/ppt/2022/6_NASA/idx2/build/Source/Python/RelWithDebInfo')
#sys.path.insert(0, 'F:/Workspace/idx2/build/Source/Python/Debug')
import os
import idx2Py as i2p
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from ipywidgets import *
import concurrent.futures
import threading
import time

path_prefix = 'J:/nasa/'
if not os.path.exists (path_prefix): 
  path_prefix = "W:/cedmav/hello/"

%matplotlib notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

vmin = -1
vmax = 1

ModuleNotFoundError: No module named 'idx2Py'

## Set up color map

In [ ]:
import colorsys

Points = [
  -10,
  -2.18552,
  -0.260014,
  0.00702716,
  0.372452,
  0.709768,
  10  
]

RGBPoints = [  
  [0.27843137254900002, 0.27843137254900002, 0.85882352941200002],
  [0.0, 0.33333333333333331, 1.0],
  [0.33333333333333331, 0.66666666666666663, 1.0],
  [0.0, 0.0, 0.49803921568627452],  
  [0.82745098039215681, 1.0, 0.73333333333333328],
  [1.0, 0.66666666666666663, 0.0],
  [0.87843137254899994, 0.30196078431399997, 0.30196078431399997]
]

def get_continuous_cmap(rgb_list, float_list=None):
  if float_list:
    # normalize the float list
    min_val = min(float_list)
    max_val = max(float_list)
    my_range = max_val - min_val
    float_list = [(v - min_val)/my_range for v in float_list]
    print(float_list)
  else:
    float_list = list(np.linspace(0, 1, len(rgb_list)))

  cdict = dict()  
  for num, col in enumerate(['red', 'green', 'blue']):
    col_list = [[float_list[i], rgb_list[i][num], rgb_list[i][num]] for i in range(0, len(float_list))]
    cdict[col] = col_list
  cmp = matplotlib.colors.LinearSegmentedColormap('my_cmp', segmentdata=cdict, N=256)
  return cmp

my_cmap = get_continuous_cmap(RGBPoints, Points)

# Some helper functions

In [ ]:
def get_file_name(input_dir, dataset, field, face, depth, time_from, time_to):
  return input_dir + '/' + dataset + '/' + field + '-face-' + repr(face) + '-depth-' + repr(depth) + '-time-' + repr(time_from) + '-' + repr(time_to) + '.idx2' 

def get_extent(dataset, face):
  if dataset == 'llc4320':
    if (face < 2):
      return (0, 0, 4320, 12960)
    elif (face > 2):
      return (0, 0, 12960, 4320)
    return (0, 0, 4320, 4320)
  elif dataset == 'llc2160':
    if (face < 2):
      return (0, 0, 2160, 6480)
    elif (face > 2):
      return (0, 0, 6480, 2160)
    return (0, 0, 2160, 2160)

# Query 
# extent should be a 4-tuple: (from_x, from_y, dim_x, dim_y)
def query_data(input_dir, dataset, times_per_file, times_per_brick, field, face, depth, time, downsampling, accuracy, extent=None):
  # determine the time step range to query from
  file_time_from    = (time // times_per_file) * times_per_file
  file_time_to      = file_time_from + times_per_file
  time_in_file      = time % times_per_file
  time_extent_from  = (time_in_file // times_per_brick) * times_per_brick
  
  # split the extent into 4 pieces
  if extent == None:
    extent = get_extent(dataset, face)
  e0 = extent
  e1 = (e0[0]           , e0[1]           , e0[2]//2   , e0[3]//2   )
  e2 = (e0[0]+(e0[2]//2), e0[1]           , e0[2]-e1[2], e0[3]//2   )
  e3 = (e0[0]           , e0[1]+(e0[3]//2), e0[2]//2   , e0[3]-e1[3])
  e4 = (e0[0]+(e0[2]//2), e0[1]+(e0[3]//2), e0[2]-e1[2], e0[3]-e1[3])
  e1 = (e1[0], e1[1], time_extent_from, e1[2], e1[3], times_per_brick)
  e2 = (e2[0], e2[1], time_extent_from, e2[2], e2[3], times_per_brick)
  e3 = (e3[0], e3[1], time_extent_from, e3[2], e3[3], times_per_brick)
  e4 = (e4[0], e4[1], time_extent_from, e4[2], e4[3], times_per_brick)

  file = get_file_name(input_dir, dataset, field, face, depth, file_time_from, file_time_to)
  print(file)
    
  array = None
  if extent[2] < 16 or extent[3] < 16: # if too small, don't spawn threads
    e = (extent[0], extent[1], time_extent_from, extent[2], extent[3], times_per_brick)
    array = i2p.DecodeExtent3f32(file, input_dir, extent, downsampling, accuracy)
  else:
    with concurrent.futures.ThreadPoolExecutor() as executor:
      f1 = executor.submit(i2p.DecodeExtent3f32, file, input_dir, e1, downsampling, accuracy)    
      f2 = executor.submit(i2p.DecodeExtent3f32, file, input_dir, e2, downsampling, accuracy)
      f3 = executor.submit(i2p.DecodeExtent3f32, file, input_dir, e3, downsampling, accuracy)
      f4 = executor.submit(i2p.DecodeExtent3f32, file, input_dir, e4, downsampling, accuracy)
      a1 = f1.result()
      a2 = f2.result()
      a3 = f3.result()
      a4 = f4.result()
      shape = (a1.shape[0], a1.shape[1] + a4.shape[1], a1.shape[2] + a4.shape[2])
      array = np.empty(shape)

      array[:, :a1.shape[1], :a1.shape[2]] = a1
      array[:, :a2.shape[1], a1.shape[2]:] = a2
      array[:, a1.shape[1]:, :a3.shape[2]] = a3
      array[:, a1.shape[1]:, a3.shape[2]:] = a4
  
  return array

# Animating four faces through time for llc2160 (1024 time steps)

In [ ]:
input_dir = path_prefix + 'llc2160_1024'
dataset = 'llc2160'
times_per_file = 1024
times_per_brick = 64
face = 0
depth = 0
accuracy = 0.1
downsampling = [2, 2, 0]

array = np.empty((1,1,1), float)
array2 = np.empty((1,1,1), float)

time_block = -1
future_time_block = 0

#future_array = [None] * 4 # for four faces
lock = threading.Lock()

def get_data(t):
  global time_block, future_time_block, future_array, array, lock
  with concurrent.futures.ThreadPoolExecutor() as executor:
      print('getting time step ', t)
      f0 = executor.submit(query_data, input_dir, dataset, times_per_file, times_per_brick, 'u', 0, depth, (t)*times_per_brick, downsampling, accuracy)    
      f1 = executor.submit(query_data, input_dir, dataset, times_per_file, times_per_brick, 'u', 1, depth, (t)*times_per_brick, downsampling, accuracy)
      f3 = executor.submit(query_data, input_dir, dataset, times_per_file, times_per_brick, 'v', 3, depth, (t)*times_per_brick, downsampling, accuracy)
      f4 = executor.submit(query_data, input_dir, dataset, times_per_file, times_per_brick, 'v', 4, depth, (t)*times_per_brick, downsampling, accuracy)
      a0 = f0.result()
      a1 = f1.result()
      a3 = f3.result()
      a4 = f4.result()
      dx = a0.shape[2]
      dy = a0.shape[1]
      dt = a0.shape[0]
      with lock:
        array = np.empty((dt, dy, dx*4), float)
        array[:, :, dx*0:dx*1] = a0[:, :, :]
        array[:, :, dx*1:dx*2] = a1[:, :, :]
        for tt in range(0, dt):
          array[tt, :, dx*2:dx*3] = np.rot90(a3[tt, :, :])
          array[tt, :, dx*3:dx*4] = np.rot90(a4[tt, :, :])
        future_time_block += 1
        
def async_get_future_data():
  global time_block, future_time_block
  #while (time_block+1) * times_per_brick < times_per_file:
  #while time_block < future_time_block:
  #time.sleep(1) # wait for the rendering to catch up 
  get_data(time_block + 1)
  
get_data(0)
#array2 = np.copy(array)
#task = threading.Thread(target=async_get_future_data)
#task.start()
fig = plt.figure(figsize = (12, 9))
plot = plt.imshow(array[0, :, :], origin = 'lower', cmap = 'bwr', aspect = 'auto', vmin = vmin, vmax = vmax)
plt.colorbar(plot)
#plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0)
fig.tight_layout()

def animate(t):
  global first_draw, array, array2, time_block, lock
  curr_time_block = t // times_per_brick
  while curr_time_block > future_time_block:
    time.sleep(1)
  
  if curr_time_block != time_block:    
    time_block = curr_time_block
    with lock:
      array2 = np.copy(array)
    task = threading.Thread(target=async_get_future_data)
    task.start()
    
  tt = min((t - time_block * times_per_brick) // pow(2, downsampling[2]), array2.shape[0]-1)
  plot.set_data(array2[tt, :,:])
  return [plot]

#interact(f, t = widgets.IntSlider(min = 0, max = 63, step = 1, value = 0, layout = Layout(width = '400px')));
anim = FuncAnimation(fig, animate, frames=1023, interval=200, blit=True)
plt.show()
